Reading the Excel file with the archaeological sites from Opole Voivodeship; cleaning and normalising the data.<br/>
File downloaded from http://www.wuozopole.pl/ochrona/rejestr-zabytkow

In [1]:
import pandas as pd
from pymongo import MongoClient

In [2]:
df = pd.read_excel('rejestr_archeologia.xls')
df.head()

,Gmina,Miejscowość,Nr stanowiska,Typ stanowiska,Chronologia,Nr rejestru zabytków,Unnamed: 6
0,Baborów,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Babice,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,16,osada,"neolit, okres wpływów rzymskich",A - 247/70,NaN
3,NaN,NaN,21,osada,neolit,A - 901/90,NaN
4,NaN,NaN,22,osada,"neolit, średniowiecze,pradzieje",A - 378/73,NaN


Remove unused column.

In [3]:
df = df[['Gmina','Miejscowość','Nr stanowiska','Typ stanowiska','Chronologia','Nr rejestru zabytków']]
df.head()

,Gmina,Miejscowość,Nr stanowiska,Typ stanowiska,Chronologia,Nr rejestru zabytków
0,Baborów,NaN,NaN,NaN,NaN,NaN
1,NaN,Babice,NaN,NaN,NaN,NaN
2,NaN,NaN,16,osada,"neolit, okres wpływów rzymskich",A - 247/70
3,NaN,NaN,21,osada,neolit,A - 901/90
4,NaN,NaN,22,osada,"neolit, średniowiecze,pradzieje",A - 378/73


Rename columns - remove Polish characters and white spaces.

In [4]:
df.rename(index=str, columns={"Miejscowość": "Miejscowosc",
                              "Nr stanowiska": "Nr_stanowiska",
                              "Typ stanowiska": "Typ_stanowiska",
                              "Nr rejestru zabytków": "Nr_rejestru_zabytkow"}, inplace = True)

Forward-fill Gmina (Municipality) and Miejscowość (Locality) columns.

In [5]:
df[['Gmina', 'Miejscowosc']] = df[['Gmina', 'Miejscowosc']].fillna(method='ffill')
df.head()

,Gmina,Miejscowosc,Nr_stanowiska,Typ_stanowiska,Chronologia,Nr_rejestru_zabytkow
0,Baborów,NaN,NaN,NaN,NaN,NaN
1,Baborów,Babice,NaN,NaN,NaN,NaN
2,Baborów,Babice,16,osada,"neolit, okres wpływów rzymskich",A - 247/70
3,Baborów,Babice,21,osada,neolit,A - 901/90
4,Baborów,Babice,22,osada,"neolit, średniowiecze,pradzieje",A - 378/73


Remove empty records.

In [7]:
df = df[pd.notna(df['Nr_stanowiska'])]
df.head()

,Gmina,Miejscowosc,Nr_stanowiska,Typ_stanowiska,Chronologia,Nr_rejestru_zabytkow
2,Baborów,Babice,16,osada,"neolit, okres wpływów rzymskich",A - 247/70
3,Baborów,Babice,21,osada,neolit,A - 901/90
4,Baborów,Babice,22,osada,"neolit, średniowiecze,pradzieje",A - 378/73
5,Baborów,Babice,23,?,"neolit, kultura łużycka, kultura przeworska",A - 372/73
6,Baborów,Babice,24,osada,"neolit, kultura celtycka",A - 428/76


Save in MongoDB.

In [8]:
client = MongoClient('localhost', 27017)
db = client['archeo']
db.sites.insert_many(df.to_dict('records'))